In [1]:
from sklearn.model_selection import ParameterGrid
import time
import numpy as np
import keras
from sklearn.metrics import classification_report

In [2]:
X_train = np.load('X_train.npy')  # Load your preprocessed training data
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')  # Load your validation data
y_val = np.load('y_val.npy')
X_test = np.load('X_test.npy')  # Load your test data
y_test = np.load('y_test.npy')

In [3]:
# Define the parameter grid
param_grid = {
    "filters": [32, 64],
    "kernel_size": [3, 5],
    "dropout_rate": [0.2, 0.4],
    "batch_size": [16, 32],
    "learning_rate": [0.001, 0.0005]
}

In [4]:
# Create a list of all parameter combinations
param_combinations = list(ParameterGrid(param_grid))
print(f"Total combinations to try: {len(param_combinations)}")

Total combinations to try: 32


In [5]:
# Limit the number of combinations to try
max_combinations = 10
param_combinations = param_combinations[:max_combinations]

# Results storage
results = []

In [6]:
num_classes = len(np.unique(y_train))

def make_model(input_shape, filters, kernel_size, dropout_rate):
    input_layer = keras.layers.Input(input_shape)
    x = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding="same")(input_layer)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(dropout_rate)(x)

    x = keras.layers.GlobalAveragePooling1D()(x)
    output_layer = keras.layers.Dense(num_classes, activation="softmax")(x)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    return model

In [7]:
best_val_acc = 0
best_model_path = "Saved Models/best_cnn.keras"

for params in param_combinations:
    print(f"\nTraining with parameters: {params}")
    start_time = time.time()

    # Create model
    model = make_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        filters=params["filters"],
        kernel_size=params["kernel_size"],
        dropout_rate=params["dropout_rate"]
    )

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=params["learning_rate"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"]
    )

    # Train the model (fewer epochs for grid search)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=8,  # Use fewer epochs for speed
        batch_size=params["batch_size"],
        verbose=1
    )

    # Evaluate on the validation set
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    elapsed_time = time.time() - start_time

    # Check if this is the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        print(f"New best model found with validation accuracy: {val_acc:.4f}")
        model.save(best_model_path)  # Save the best model

    # Store the results
    results.append({
        "params": params,
        "val_loss": val_loss,
        "val_acc": val_acc,
        "time": elapsed_time
    })

    print(f"Validation Accuracy: {val_acc:.4f} | Time: {elapsed_time:.2f}s")


Training with parameters: {'batch_size': 16, 'dropout_rate': 0.2, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.001}
Epoch 1/8
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.6185 - sparse_categorical_accuracy: 0.6981 - val_loss: 0.4836 - val_sparse_categorical_accuracy: 0.7579
Epoch 2/8
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.4429 - sparse_categorical_accuracy: 0.8484 - val_loss: 0.4070 - val_sparse_categorical_accuracy: 0.8684
Epoch 3/8
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.4118 - sparse_categorical_accuracy: 0.8432 - val_loss: 0.3869 - val_sparse_categorical_accuracy: 0.8722
Epoch 4/8
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.4126 - sparse_categorical_accuracy: 0.8423 - val_loss: 0.3822 - val_sparse_categorical_accuracy: 0.8732
Epoch 5/8
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.4059 - sparse_categorical_accuracy: 0.8345 - val_loss: 0.3765 - val_sparse_categorical_accuracy: 0.8742
Epoch 6/8
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms

In [8]:
# Load the saved best model
best_model = keras.models.load_model(best_model_path)

# Generate predictions on the test set
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

# Generate and print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.87      0.91       539
           1       0.79      0.92      0.85       298

    accuracy                           0.89       837
   macro avg       0.87      0.89      0.88       837
weighted avg       0.90      0.89      0.89       837

